In [1]:
import random
from importlib import reload

import torch

from datasets import load_dataset

from transformers import AutoTokenizer, AutoModelForCausalLM

from util import mt

In [2]:
model_name = 'EleutherAI/gpt-neo-125M'
model_name = 'facebook/xglm-7.5B'
model_name = 'facebook/xglm-4.5B'
model_name = 'facebook/xglm-2.9B'
model_name = 'facebook/xglm-564M'

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).eval().half().cuda()

In [192]:
src = load_dataset("gsarti/flores_101", 'kor')
tgt = load_dataset("gsarti/flores_101", 'deu')
eng = load_dataset("gsarti/flores_101", 'eng')

Found cached dataset flores_101 (/gscratch/zlab/ahai/hf/datasets/gsarti___flores_101/kor/1.0.0/e663cc717b274f2cef5142df786973abbf1966a7115d99509c062936d77d4335)


  0%|          | 0/2 [00:00<?, ?it/s]

Generating dev split:   0%|          | 0/997 [00:00<?, ? examples/s]

Generating devtest split:   0%|          | 0/1012 [00:00<?, ? examples/s]

Dataset flores_101 downloaded and prepared to /gscratch/zlab/ahai/hf/datasets/gsarti___flores_101/deu/1.0.0/e663cc717b274f2cef5142df786973abbf1966a7115d99509c062936d77d4335. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Found cached dataset flores_101 (/gscratch/zlab/ahai/hf/datasets/gsarti___flores_101/eng/1.0.0/e663cc717b274f2cef5142df786973abbf1966a7115d99509c062936d77d4335)


  0%|          | 0/2 [00:00<?, ?it/s]

In [209]:
src_lang = 'Korean'
tgt_lang = 'Deutsch'

In [5]:
def tokenize(s):
  ids = []
  i = 0
  t = 0
  while t < len(s):
    c = s[t]
    if c == '\n':
      ids += s_ids(s[i:t]) + [2]
      i = t+1
      t = t+1
    else:
      t += 1
  ids += s_ids(s[i:])
  return ids

In [6]:
def s_ids(s):
  return tokenizer(s).input_ids[1:]

In [7]:
def make_demo_prompt(src_endonym, tgt_endonym, src, tgt, k):
  src_points = random.sample(list(src), k=k)
  tgt_points = [ tgt[point['id']-1] for point in src_points ]
  demos = []
  for s, t in zip(src_points, tgt_points):
    demo = f"{src_endonym} : {s['sentence']} {tgt_endonym} : {t['sentence']}"
    demos.append(demo)
  demo_prompt_ids = [2]
  for demo in demos:
    demo_prompt_ids.extend(s_ids(demo))
    demo_prompt_ids.append(2)
  return demo_prompt_ids

In [146]:
reload(mt)

<module 'util.mt' from '/mmfs1/gscratch/zlab/ahai/repo/infogain/util/mt.py'>

In [194]:
src_points = random.sample(list(src['devtest']), k=10)
tgt_points = [ tgt['devtest'][point['id']-1] for point in src_points ]

In [210]:
demo_prompt_ids = mt.make_demo_prompt(src_points, tgt_points, src_lang, tgt_lang, tokenizer)
uncond_demo_prompt_ids = mt.make_uncond_demo_prompt(tgt_points, tokenizer)

In [211]:
idx = 512
print(src['dev'][idx]['sentence'])
print()
print(tgt['dev'][idx]['sentence'])
print()
print(eng['dev'][idx]['sentence'])

ADD는 다른 또래와의 관계에 영향을 미치는데, 이는 다른 아이들은 ADD 증상이 있는 아이들이 왜 그렇게 행동하는지, 왜 그렇게 말하는지, 또는 성숙도가 왜 차이가 나는지 이해할 수 없기 때문입니다.

ADHS beeinträchtigt Beziehungen zu Gleichaltrigen, weil andere Kinder nicht verstehen können, warum sie sich so verhalten, warum sie so buchstabieren oder warum ihr Reifegrad anders ist.

ADD affects relationships with other peers because other children can not understand why they act the way that they do or why they spell they way they do or that their maturity level is different.


In [213]:
inference_point = src['dev'][idx] 
cond_prompt_ids = demo_prompt_ids + s_ids(f"{src_lang} : {inference_point['sentence']} {tgt_lang} :")
cond_prompt_t = torch.LongTensor([cond_prompt_ids]).to(model.device)
uncond_prompt_ids = demo_prompt_ids + s_ids(f"{src_lang} : N/A {tgt_lang} :")
uncond_prompt_t = torch.LongTensor([uncond_prompt_ids]).to(model.device)
#uncond_prompt_t = torch.LongTensor([uncond_demo_prompt_ids]).to(model.device)

In [214]:
with torch.no_grad():
  gen_ids = mt.cfg_greedy_explore(model, 0.3, cond_prompt_t, uncond_prompt_t, 50, tokenizer)
tokenizer.decode(gen_ids[0].tolist())

138919 | ADD ADH 이것은 예 이러한 0.82275390625| ADD 이  ADH 예 0.306396484375
557 | 는가의 아동에 0.74072265625| 는가의와를 0.72705078125
9024 | 다른 또 ADH 성  0.43701171875| 다른  또 성 ADH 0.2802734375
19068 | 또 아이들 사람들 학생들 다른 0.99072265625| 또 아이들 사람들 사람 사람들이 0.89892578125
7623 | 래 다른레랫뇌 1.0| 래 다른레랫<unk> 1.0
1351 | 와들과 아이들의에 0.84765625| 와들과의 아이들에 0.78564453125
277 | 의 관계  다른 의사 0.97021484375| 의 다른  관계는 0.931640625
37510 | 관계 상 연  연결 0.9912109375| 관계 상  연 접 0.95703125
428 | 에에서에도를가 0.9873046875| 에에서를가에도 0.9677734375
83035 | 영향을 영향 많은 대해 중요한 0.99853515625| 영향을 영향 대해 대한 많은 0.99072265625
5877 | 미 </s> 받 주 0.99853515625| 미  받 주 줄 0.99658203125
3271 | 치치는칠쳐친 0.998046875| 치치는칠쳐친 0.99658203125
16301 | 는데데지만는데요기에 0.99951171875| 는데지만지기고 0.994140625
4 | ,.( ( 이는 0.9990234375| , 이는.( ( 0.99609375
69617 | 이는 이것은 그것은 이러한 이 0.99951171875| 이는 이것은 그것은 이러한 이 0.99853515625
9024 | 다른 또다른 같은 어떤 0.99951171875| 다른 또 같은 더 그 0.99609375
94586 | 아이들 아이 학생들 kids아이 1.0| 아이들 아이 학생들 또 사람들이 0.99853515625
555 | 은이 왜도과 1.0| 은이과도의 0.998046875


'ADD는 다른 또래와의 관계에 영향을 미치는데, 이는 다른 아이들은 ADD 증상이 있는 아이들이 왜 그렇게 행동하는지, 왜 그렇게 말하는지, 또는 성숙도가 왜 차이가 나는지 이해할 수 없기 때문'

In [215]:
with torch.no_grad():
  greedy_gen_ids = mt.greedy(model, cond_prompt_t, 50)
tokenizer.decode(greedy_gen_ids[0].tolist())

'ADD는 다른 또래와의 관계에 영향을 미치는데, 이는 다른 아이들은 ADD 증상이 있는 아이들이 왜 그렇게 행동하는지, 왜 그렇게 말하는지, 또는 성숙도가 왜 차이가 나는지 이해할 수 없기 때문'

In [207]:
tokenizer.decode(demo_prompt_ids)

'</s> Korean : 해외에 거주하다 귀국할 경우, 타국의 문화에 적응한 동시에 자국의 문화의 일부 습관을 잃는 모습을 보이기도 한다. German : Wenn man nach einem Auslandsaufenthalt nach Hause kommt, hat man sich an eine andere Kultur angepasst und einige Verhaltensweisen seiner Heimatkultur abgelegt.</s></s> Korean : 만약 작지만 풍미가 진한 <unk>을 원한다면, 지역에 따라 베를리너, <unk>쿠<unk>, 또는 크라펜으로 불리는 이 <unk>을 드셔보세요. German : Wenn Sie kleine, aber reichhaltige Backwaren herstellen wollen, dann probieren Sie das, was je nach Region als Berliner, Pfannkuchen oder Krapfen bezeichnet wird.</s></s> Korean : 그들은 해가 다른 별들과 같이 발광성과 회전의 원리만으로 작동한다는 것을 발견했습니다. German : Sie haben herausgefunden, dass die Sonne auf Basis derselben Grundprinzipien funktioniert, wie andere Sterne: Es wurde herausgefunden, dass die Aktivität aller Sterne im System durch ihre Leuchtkraft, Rotation und sonst nichts angetrieben wird.</s></s> Korean : "<unk>버스는 ""수백만의 지구인들에게 널리 가해진 죽음, 파괴 및 수백만 번의 테러""에 대해 신을 고소했습니다." German : Chambers verklagte Gott wegen „weitverbreitetem Tod, Zerstörung und

In [113]:
prompt_ids = torch.LongTensor([tokenizer('translate into english, 中文： 谢谢你！').input_ids[:-1]]).to(model.device)

In [232]:
prompt_ids = torch.LongTensor([tokenize('中文： 谢谢你! english:')]).to(model.device)                          

In [233]:
prompt_ids.tolist()

[[6, 19773, 13, 6, 13305, 13305, 710, 35, 83763, 13]]

In [351]:
model.config

XGLMConfig {
  "_name_or_path": "facebook/xglm-2.9B",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": [
    "XGLMForCausalLM"
  ],
  "attention_dropout": 0.1,
  "attention_heads": 16,
  "bos_token_id": 0,
  "d_model": 2048,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "eos_token_id": 2,
  "ffn_dim": 8192,
  "init_std": 0.02,
  "layerdrop": 0.0,
  "max_position_embeddings": 2048,
  "model_type": "xglm",
  "num_layers": 48,
  "pad_token_id": 1,
  "scale_embedding": true,
  "transformers_version": "4.22.2",
  "use_cache": true,
  "vocab_size": 256008
}

In [275]:
with torch.no_grad():
  gen_ids = mt.sample(model, prompt_ids, 200)

TypeError: expected Tensor as element 0 in argument 0, but got list

In [309]:
tokenizer.decode(gen_ids[0].tolist())

'Leading researchers say that the test can detect cancer, tuberculosis, HIV and malaria in patients who live in low-income countries, where for example, the survival rate of breast cancer is half that of the richer countries.</s> Finnish.</s> Finnish : Hän sanoi, että hän sanoi, että hän sanoi, että, että, että, että, että, että, että, että, että, että, että, että, että, että, että, että,'

In [231]:
print(tokenizer.decode(prompt_ids[0]))

中文: 谢谢你! | english:


In [14]:
inference_point['sentence']

'Stanfordin yliopiston lääketieteen laitoksen tutkijat ilmoittivat maanantaina uuden diagnostiikkatyökalun keksimisestä: solut tyypin mukaan lajitteleva pienenpieni tulostettava siru, joka voidaan valmistaa normaaleilla mustesuihkutulostimilla mahdollisesti noin yhden Yhdysvaltain sentin kappalehintaan.'

In [209]:
s_ids('')

[]

In [219]:
def tokenize(s):
  ids = []
  i = 0
  t = 0
  while t < len(s):
    c = s[t]
    if c == '\n':
      ids += s_ids(s[i:t]) + [2]
      i = t+1
      t = t+1
    else:
      t += 1
  ids += s_ids(s[i:])
  return ids

In [12]:
vocab = tokenizer.get_vocab()
tokens = list(sorted(tokenizer.get_vocab(), key=lambda k: vocab[k]))

In [121]:
tokens[:15]

['<s>',
 '<pad>',
 '</s>',
 '<unk>',
 ',',
 '.',
 '▁',
 's',
 '-',
 'a',
 '▁de',
 '▁a',
 'e',
 ':',
 'i']

In [305]:
dir(tokenizer)

['SPECIAL_TOKENS_ATTRIBUTES',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_tokens',
 '_additional_special_tokens',
 '_auto_class',
 '_batch_encode_plus',
 '_bos_token',
 '_call_one',
 '_cls_token',
 '_convert_encoding',
 '_convert_id_to_token',
 '_convert_token_to_id_with_added_voc',
 '_create_repo',
 '_decode',
 '_decode_use_source_tokenizer',
 '_encode_plus',
 '_eos_token',
 '_eventual_warn_about_too_long_sequence',
 '_eventually_correct_t5_max_length',
 '_from_pretrained',
 '_get_files_timestamps',
 '_get_padding_truncation_strategies',
 '_in_target_context_manager',
 '_mask_token',
 '_pad',
 '_pad_token',
 '_pad

In [306]:
tokenizer.model_max_length

1000000000000000019884624838656

In [319]:
def make_demo_prompt(src_points, tgt_points, src_name, tgt_name, src, tgt, k, tokenizer):
    demos = []
    for s, t in zip(src_points, tgt_points):
        assert(s['id'] == t['id'])
        demo = f"{src_name} : {s['sentence']} {tgt_name} : {t['sentence']}"
        demos.append(demo)
    demo_prompt_ids = [2]
    for demo in demos:
      demo_prompt_ids.extend(mt.xglm_tokenize(demo, tokenizer))
      demo_prompt_ids.append(2)
    return demo_prompt_ids

In [323]:
src_points = random.sample(list(src['dev']), k=3)
tgt_points = [ tgt['dev'][point['id']-1] for point in src_points ]

In [334]:
reload(mt)

<module 'util.mt' from '/mmfs1/gscratch/zlab/ahai/repo/infogain/util/mt.py'>

In [330]:
print(mt.post_proc(tokenizer.decode(mt.make_demo_prompt(src_points, tgt_points, 'finnish', 'english', tokenizer))))


 finnish : Kiertoajelut ovat halvempia suurille ryhmille, joten jos liikut yksin tai vain yhden ystävän kanssa, koeta tutustua muihin ihmisiin ja muodostaa 4–6 hengen ryhmä parempaa yksilöhintaa varten. english : Tours are cheaper for larger groups, so if you're by yourself or with just one friend, try to meet other people and form a group of four to six for a better per-person rate.
 finnish : Vaatimukset on tarkoitettu tekemään maiden välisestä muuttovirrasta järjestelmällisen. english : These requirements are designed to provide an organized migratory flow between both countries.
 finnish : Varo: pikkukaupunkien baarit eivät täälläpäin aina ole parhaita ajanviettopaikkoja vieraspaikkakuntalaisille. english : Beware: small-town bars here are not always good places for the out-of-state visitor to hang out.



In [331]:
def make_uncond_demo_prompt(tgt_points, tokenizer):
    uncond_demo_prompt_ids = [2]
    for t in tgt_points:
      uncond_demo_prompt_ids.extend(mt.xglm_tokenize(t['sentence'], tokenizer))
      uncond_demo_prompt_ids.append(2)
    return uncond_demo_prompt_ids

In [333]:
print(mt.post_proc(tokenizer.decode(make_uncond_demo_prompt(tgt_points, tokenizer))))


 Tours are cheaper for larger groups, so if you're by yourself or with just one friend, try to meet other people and form a group of four to six for a better per-person rate.
 These requirements are designed to provide an organized migratory flow between both countries.
 Beware: small-town bars here are not always good places for the out-of-state visitor to hang out.

